# Simulation Framework

In [1]:
import subprocess
import pandas as pd
import numpy as np
import json
import argparse
from pathlib import Path
import shutil
from typing import Dict, List, Tuple

In [6]:
vcf_file = Path("../data/raw/geno_phased_fixed.vcf.gz")
output_dir = Path("../data/processed/simulationx")
n_samples = 2500

## Genotyping Data Conversion 

Converts VCF genotype file to PLINK format, and randomly samples n rows from this file. 

In [7]:
temp_prefix = output_dir / "temp_all"
cmd = f"""
plink2 --vcf {vcf_file} \
        --make-bed \
        --out {temp_prefix}
"""
subprocess.run(cmd, shell=True, check=True)

# read fam file to get sample info 
fam = pd.read_csv(f"{temp_prefix}.fam", sep=r'\s+', header=None,
                    names=['fid', 'iid', 'father', 'mother', 'sex', 'pheno'])

# select n random samples
n_available = len(fam)
n_to_select = min(n_samples, n_available)
selected_indices = np.random.choice(n_available, n_to_select, replace=False)
selected_samples = fam.iloc[selected_indices]

# Create keep file with FID and IID columns
samples_file = output_dir / "samples_keep.txt"
selected_samples[['fid', 'iid']].to_csv(samples_file, sep='\t', header=False, index=False)

# create plink clean directory
plink_clean_dir = output_dir / "plink_clean"
plink_clean_dir.mkdir(exist_ok=True)
plink_prefix = plink_clean_dir / "chr22_data"

cmd = f"""
plink2 --bfile {temp_prefix} \
        --keep {samples_file} \
        --make-bed \
        --maf 0.01 \
        --geno 0.05 \
        --out {plink_prefix}
"""
subprocess.run(cmd, shell=True, check=True)

# clean up temp files 
for ext in ['.bed', '.bim', '.fam', '.log']:
    temp_file = Path(f"{temp_prefix}{ext}")
    if temp_file.exists():
        temp_file.unlink()

plink_prefix = str(plink_prefix)

PLINK v2.0.0-a.7 M1 (20 Sep 2025)                   cog-genomics.org/plink/2.0/
(C) 2005-2025 Shaun Purcell, Christopher Chang    GNU General Public License v3
Logging to ../data/processed/simulationx/temp_all.log.
Options in effect:
  --make-bed
  --out ../data/processed/simulationx/temp_all
  --vcf ../data/raw/geno_phased_fixed.vcf.gz

Start time: Wed Oct 29 21:04:46 2025
16384 MiB RAM detected; reserving 8192 MiB for main workspace.
Using up to 10 threads (change this with --threads).
--vcf: 1070401 variants scanned.
--vcf: ../data/processed/simulationx/temp_all-temporary.pgen +
../data/processed/simulationx/temp_all-temporary.pvar.zst +
../data/processed/simulationx/temp_all-temporary.psam written.
3202 samples (0 females, 0 males, 3202 ambiguous; 3202 founders) loaded from
../data/processed/simulationx/temp_all-temporary.psam.
1070401 variants loaded from
../data/processed/simulationx/temp_all-temporary.pvar.zst.
Note: No phenotype data present.
Writing ../data/processed/simulatio

## Genotype HDF5 conversion

Genotype files are converted to HDF5 format for efficient access.

In [8]:
convert_dir = output_dir / "gennet_convert_temp"
if convert_dir.exists():
    shutil.rmtree(convert_dir)
convert_dir.mkdir()

# copy only plink files to the directory 
base_name = Path(plink_prefix).name
for ext in ['.bed', '.bim', '.fam']:
    src = Path(f"{plink_prefix}{ext}")
    dst = convert_dir / f"{base_name}{ext}"
    shutil.copy2(src, dst)

h5_output_dir = output_dir / "h5_output"
if h5_output_dir.exists():
    shutil.rmtree(h5_output_dir)
h5_output_dir.mkdir()

gennet_script = Path("../GenNet.py")

cmd = f"""python {gennet_script} convert \
    -g {convert_dir} \
    -study_name {base_name} \
    -o {h5_output_dir}"""

subprocess.run(cmd, shell=True, check=True)

if convert_dir.exists():
    shutil.rmtree(convert_dir)

using ../data/processed/simulationx/h5_output/
Number of Individuals: 2500
Number of Probes 244268 in chr22_data.bim
Number of Probes 244268 converted
next 25000 SNPs, from 244268, need to convert 219268
Time to read 25000 SNPs is 1.6196000576019287 s
Time to write 25000 SNPs is 12.224303960800171 s
next 25000 SNPs, from 244268, need to convert 194268
Time to read 25000 SNPs is 1.6037170886993408 s
Time to write 25000 SNPs is 11.986586809158325 s
next 25000 SNPs, from 244268, need to convert 169268
Time to read 25000 SNPs is 1.592545747756958 s
Time to write 25000 SNPs is 11.025698184967041 s
next 25000 SNPs, from 244268, need to convert 144268
Time to read 25000 SNPs is 1.8886241912841797 s
Time to write 25000 SNPs is 12.821829080581665 s
next 25000 SNPs, from 244268, need to convert 119268
Time to read 25000 SNPs is 1.5810661315917969 s
Time to write 25000 SNPs is 11.924237966537476 s
next 25000 SNPs, from 244268, need to convert 94268
Time to read 25000 SNPs is 1.5950350761413574 s


  0%|          | 0/121 [00:00<?, ?it/s]

merged shape = (244268, 2500)
probe shape = (244268, 6)

 impute missing...
2046


  0%|          | 0/3 [00:00<?, ?it/s]

WARNING skipped step 4, all variants are used: using ../data/processed/simulationx/h5_output//chr22_data_step3_genotype_no_missing.h5
chuncksize = 2046


100%|██████████| 3/3 [00:10<00:00,  3.55s/it]


Completed chr22_data
You can delete all other h5 files if genotype.h5 is correct


## Topology Matrix
Create an annotation matrix to define partial connections between layers

In [ ]:

# Read BIM file
bim = pd.read_csv(f"{plink_prefix}.bim", sep='\t', header=None,
                    names=['chr', 'snp', 'cm', 'pos', 'a1', 'a2'])

# Create gene to pathway mapping
n_pathways = max(1, n_genes // 20)  # ~20 genes per pathway
gene_to_pathway = {}
pathway_idx = 0
for gene_idx in range(n_genes):
    if gene_idx > 0 and gene_idx % 20 == 0:
        pathway_idx += 1
    gene_to_pathway[gene_idx] = pathway_idx

topology_rows = []
for snp_idx, row in bim.iterrows():
    # Assign SNP to gene (distribute evenly)
    gene_idx = snp_idx % n_genes
    pathway_idx = gene_to_pathway[gene_idx]

    topology_rows.append({
        'chr': row['chr'],
        'layer0_node': snp_idx,
        'layer0_name': row['snp'],
        'layer1_node': gene_idx,
        'layer1_name': f"GENE_{gene_idx}",
        'layer2_node': pathway_idx,
        'layer2_name': f"PATHWAY_{pathway_idx}"
    })

topology_file = self.output_dir / "topology.csv"
pd.DataFrame(topology_rows).to_csv(topology_file, index=False)

print(f"Topology created: {len(topology_rows)} SNPs -> {n_genes} genes -> {len(set(gene_to_pathway.values()))} pathways")

return str(topology_file), topology_rows
    